In [76]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import SGDClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import cross_validation

In [67]:
allData = pd.read_csv('../assignment1_2016S1/training_data.csv',header=None).sort_values(by=0).reset_index().ix[:10000,2:]
allLabels = pd.read_csv('../assignment1_2016S1/training_labels.csv',header=None).sort_values(by=0).reset_index().ix[:10000,2:]

In [68]:
#svd = TruncatedSVD(n_components=1000, random_state=42)
#pcaData = svd.fit_transform(allData)

In [71]:
dataMatrix = allData.as_matrix()
labelMatrix = allLabels.as_matrix()
kf = KFold(allData.shape[0],20)
for train_index,test_index in kf:
    X_train,X_test = dataMatrix[train_index],dataMatrix[test_index]
    y_train,y_test = labelMatrix[train_index],labelMatrix[test_index]

In [72]:
((X_train.shape,y_train.shape),(X_test.shape,y_test.shape))

(((9501, 13626), (9501, 1)), ((500, 13626), (500, 1)))

# Gaussian Naive Bayes

In [73]:
clf = GaussianNB().fit(X_train,y_train.ravel())
clf.score(X_test,y_test)

0.376

# Multinomial Naive Bayes

In [74]:
clf = MultinomialNB().fit(X_train,y_train.ravel())
clf.score(X_test,y_test)

0.60999999999999999

# Decision Tree

In [75]:
clf = tree.DecisionTreeClassifier().fit(X_train,y_train.ravel())
clf.score(X_test,y_test)

0.41799999999999998

# SVM - Linear Kernel

In [78]:
clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42).fit(X_train,y_train.ravel())
clf.score(X_test,y_test)

C:\Users\rafae\Anaconda3\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.59599999999999997

# Random Forest

In [36]:
clf = RandomForestClassifier(n_estimators=100).fit(X_train,y_train.ravel())
clf.score(X_test,y_test)

0.38

(951, 1)

0.16